# Implement the FP-Growth algorithm for association rule mining and post your code in GitHub.

In [1]:
from itertools import chain, combinations
from collections import defaultdict, OrderedDict

class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

            
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    fpTree = Node('Null', 1, None)
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable


def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

        
def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        treeNode.children[item].increment(frequency)
    else:
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]


def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

def findPrefixPath(basePat, headerTable):
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        treeNode = treeNode.next  
    return condPats, frequency


def mineTree(headerTable, minSup, preFix, freqItemList):
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    for item in sortedItemList:  
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print("No frequent item set found.")
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

In [2]:
itemSetList = [['bread', 'milk'],
                ['bread', 'diaper', 'beer', 'eggs'],
                ['milk', 'diaper', 'beer', 'coke'],
                  ['bread', 'milk', 'diaper', 'beer'],
                  ['bread', 'milk', 'diaper', 'coke']]
freqItemSet, rules = fpgrowth(itemSetList, minSupRatio=0.4, minConf=0.6)
print("Frequent itemset:", freqItemSet)
print("Association rules generated:", rules) 

Frequent itemset: [{'coke'}, {'coke', 'diaper'}, {'coke', 'milk'}, {'coke', 'diaper', 'milk'}, {'beer'}, {'beer', 'bread'}, {'beer', 'bread', 'diaper'}, {'beer', 'milk'}, {'beer', 'diaper', 'milk'}, {'beer', 'diaper'}, {'bread'}, {'milk'}, {'bread', 'milk'}, {'diaper'}, {'bread', 'diaper'}, {'bread', 'diaper', 'milk'}, {'diaper', 'milk'}]
Association rules generated: [[{'coke'}, {'diaper'}, 1.0], [{'coke'}, {'milk'}, 1.0], [{'coke'}, {'diaper', 'milk'}, 1.0], [{'coke', 'diaper'}, {'milk'}, 1.0], [{'coke', 'milk'}, {'diaper'}, 1.0], [{'diaper', 'milk'}, {'coke'}, 0.6666666666666666], [{'beer'}, {'bread'}, 0.6666666666666666], [{'beer'}, {'bread', 'diaper'}, 0.6666666666666666], [{'beer', 'bread'}, {'diaper'}, 1.0], [{'beer', 'diaper'}, {'bread'}, 0.6666666666666666], [{'bread', 'diaper'}, {'beer'}, 0.6666666666666666], [{'beer'}, {'milk'}, 0.6666666666666666], [{'beer'}, {'diaper', 'milk'}, 0.6666666666666666], [{'beer', 'diaper'}, {'milk'}, 0.6666666666666666], [{'beer', 'milk'}, {'dia